# **Simple Titanic Model Using Tensorflow**
## **Used Library**
* **Tensorflow**
* **numpy**
* **pandas**

### **This is my first competition & notebook!**
#### **Recently, I got interested in Machine-Learning & Deep-Learning**
* ****↓↓↓ Visit my first project using NEAT Algorithm!! ↓↓↓****
* [My Github Homepage](https://github.com/JunHyeok-Navy/NEAT_Ship_Simulator)

#### Reference
* Titanic Tutorial
* Youtube


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
train_data.head()

In [ ]:
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
test_data.head()

In [ ]:
women = train_data.loc[train_data.Sex == 'female']['Survived']
rate_women = sum(women)/len(women)

print('rate of survival number',  rate_women)

In [ ]:
men = train_data.loc[train_data.Sex == 'male']['Survived']
rate_men = sum(men)/len(men)

print('rate pf survival number', rate_men)

In [ ]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
import math

y = train_data['Survived'].to_numpy()
y_1 = y
features = ['Pclass', 'Sex', 'SibSp', 'Parch']
x_train = pd.get_dummies(train_data[features]).to_numpy()
x_train_1 = x_train
x_test = pd.get_dummies(test_data[features]).to_numpy()
ratio = math.trunc(len(x_train)*0.8)
x_train = x_train[:ratio]
x_valid = x_train_1[ratio:]
y_train = y[:ratio]
y_valid = y_1[ratio:]

model = Sequential([
    Dense(1024, input_shape = (5,), activation = 'relu'),
    Dropout(0.5),
    Dense(512, activation = 'relu'),
    Dropout(0.5),
    Dense(256, activation = 'relu'),
    Dropout(0.5),
    Dense(256, activation = 'relu'),
    Dense(128, activation = 'relu'),
    Dropout(0.5),
    Dense(64, activation = 'relu'),
    Dense(32, activation = 'relu'),
    Dropout(0.5),
    Dense(16, activation = 'relu'),
    Dense(1, activation = 'sigmoid')
])
model.compile(optimizer = 'adam', loss='binary_crossentropy', metrics = ['acc'])
c_path = 'check.ckpt'
checkpoint = ModelCheckpoint(filepath = c_path,
                             save_weights_only=True,
                             verbose=1,
                             save_best_only = True,
                             monitor = 'val_loss'
                            )
model.fit(x_train, y_train, validation_data=(x_valid, y_valid), epochs=100,
          callbacks=[checkpoint], verbose = 1)
model.load_weights(c_path)
predictions = model.predict(x_test)

print(predictions)

### I got values of 'predictions' from 0 to 1.
#### I need to convert these values to Survived value by own basis (prediction which is larger than 0.5 → save Survived value to 1)

In [ ]:
pre_label = []
for i in range(len(predictions)):
    if predictions[i][0] >= 0.5:
        pre_label.append(1)
    elif predictions[i][0] < 0.5:
        pre_label.append(0)
output = pd.DataFrame({'PassengerId' : test_data.PassengerId,
                      'Survived' : pre_label})
print(output)
# output.to_csv('my_submission_2.csv', index=False)